In [11]:
import pandas as pd
import numpy as np

# Process

In [15]:
#read the excel file after importing it to the collab
df=pd.read_csv('/content/drive/MyDrive/GUVI/GUVI Capstone 2/data analytics (2).csv')

In [16]:
#to know the number of rows and columns
df.shape


(532958, 13)

In [17]:
# to know the datatype and number of non null count of rows in each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532958 entries, 0 to 532957
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             532958 non-null  object 
 1   rideable_type       532958 non-null  object 
 2   started_at          532958 non-null  object 
 3   ended_at            532958 non-null  object 
 4   start_station_name  513267 non-null  object 
 5   start_station_id    513057 non-null  float64
 6   end_station_name    509585 non-null  object 
 7   end_station_id      509434 non-null  float64
 8   start_lat           532958 non-null  float64
 9   start_lng           532958 non-null  float64
 10  end_lat             532169 non-null  float64
 11  end_lng             532169 non-null  float64
 12  member_casual       532958 non-null  object 
dtypes: float64(6), object(7)
memory usage: 52.9+ MB


In [18]:
#to know the basic statistical info of the columns
df.describe()

,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng
count,513057.000000,509434.000000,532958.000000,532958.000000,532169.000000,532169.000000
mean,223.603798,224.199541,41.904237,-87.644660,41.904565,-87.644973
std,162.208577,162.142616,0.043290,0.025573,0.043439,0.025787
min,2.000000,2.000000,41.651867,-87.840000,41.640000,-87.880000
25%,94.000000,94.000000,41.882664,-87.658617,41.882830,-87.658865
50%,194.000000,194.000000,41.900000,-87.641066,41.900680,-87.641697
75%,308.000000,309.000000,41.929567,-87.627733,41.930000,-87.627734
max,721.000000,721.000000,42.080000,-87.520000,42.120000,-87.520000


# Cleaning

In [19]:
#create a new dataframe with the same file to continue the cleaning process
df1=df
# TO KNOW THE NUMBER OF MISSING ROWS IN STATION NAME
null_start_stations=df1['start_station_name'].isnull().sum()
null_end_stations=df1['end_station_name'].isnull().sum()
print(f"Number of null values in start_station_name: {null_start_stations}")
print(f"Number of null values in end_station_name: {null_end_stations}")

Number of null values in start_station_name: 19691
Number of null values in end_station_name: 23373


In [20]:
# DROP THE ROWS WITH MISSING STATION NAME
df2=df1.dropna(subset=['start_station_name','end_station_name'])


In [21]:
#df2 is our new dataframe with no null values in station names
df2.shape

(500684, 13)

Its acceptable to lose max 10% of data while doing data cleaning.

In [22]:
diff = abs(len(df1.index) - len(df2.index))
percent_lost = (diff / len(df1.index)) * 100

print(f"Number of rows lost after dropping null values: {diff}")
print(f'Percent of rows lost after dropping null values: {percent_lost:.2f}%')


Number of rows lost after dropping null values: 32274
Percent of rows lost after dropping null values: 6.06%


In [23]:
# finding error like the end time of the ride being lesser than the start time
df2[(df2['started_at'] > df2['ended_at'])][['started_at', 'ended_at']]

,started_at,ended_at
637,30-09-2020 23:46,01-10-2020 00:56
2235,30-09-2020 23:51,01-10-2020 00:01
3122,30-09-2020 23:48,01-10-2020 00:05
3124,30-09-2020 22:48,01-10-2020 00:06
3125,30-09-2020 23:53,01-10-2020 01:17
...,...,...
515338,04-09-2020 19:52,04-09-2020 19:51
519923,30-09-2020 23:52,01-10-2020 01:17
520721,30-09-2020 23:58,01-10-2020 00:08
530078,30-09-2020 23:46,01-10-2020 00:21


# Create a temporary column to hold the values of the first column
df['temp'] = df['column_1']

# Assign the values of the second column to the first column where the condition is true
df.loc[condition, 'column_1'] = df.loc[condition, 'column_2']

# Assign the values of the temporary column to the second column
df.loc[condition, 'column_2'] = df['temp']

# Drop the temporary column
df.drop(columns=['temp'], inplace=True)

This solution uses a temporary column to hold the values of the first column while the values of the two columns are being exchanged. This prevents data loss.

Here's a breakdown of the solution:

Create a temporary column: This creates a new column called temp that holds the values of column_1.
Assign values based on condition: This uses the loc method to assign the values of column_2 to column_1 for rows where the condition is true.
Assign values from temporary column: This assigns the values from the temp column to column_2 for rows where the condition is true.
Drop temporary column: This drops the temp column that was used to hold the values during the exchange.
Note: Replace column_1, column_2, and condition with the actual names and condition for your specific problem.

In [24]:
df2['temp'] = df2['started_at']


<ipython-input-24-bb436ad9866c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['temp'] = df2['started_at']


In [25]:
df2.loc[(df2['started_at'] > df2['ended_at']), 'started_at'] = df2.loc[(df2['started_at'] > df2['ended_at']), 'ended_at']
df2.loc[(df2['started_at'] > df2['ended_at']), 'ended_at'] = df2['temp']
df2.drop(columns=['temp'], inplace=True)


<ipython-input-25-47904a41ca77>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns=['temp'], inplace=True)


In [26]:
df2[(df2['started_at'] > df2['ended_at'])]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual


Create a new column to know the duration of each ride.

In [28]:
# Convert 'started_at' and 'ended_at' columns to datetime
df2['started_at'] = pd.to_datetime(df2['started_at'])
df2['ended_at'] = pd.to_datetime(df2['ended_at'])

# Calculate the duration
df2['Duration'] = df2['ended_at'] - df2['started_at']

# Display the first five rows
df2.head(5)

<ipython-input-28-600629d8cc8e>:2: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df2['started_at'] = pd.to_datetime(df2['started_at'])
<ipython-input-28-600629d8cc8e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['started_at'] = pd.to_datetime(df2['started_at'])
<ipython-input-28-600629d8cc8e>:3: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df2['ended_at'] = pd.to_datetime(df2['ended_at'])
<ipython-input-28-600629d8cc8e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Duration
0,2B22BD5F95FB2629,electric_bike,2020-09-17 14:27:00,2020-09-17 14:44:00,Michigan Ave & Lake St,52.0,Green St & Randolph St,112.0,41.886692,-87.623561,41.883570,-87.648731,casual,0 days 00:17:00
1,A7FB70B4AFC6CAF2,electric_bike,2020-09-17 15:07:00,2020-09-17 15:07:00,W Oakdale Ave & N Broadway,NaN,W Oakdale Ave & N Broadway,NaN,41.940000,-87.640000,41.940000,-87.640000,casual,0 days 00:00:00
2,86057FA01BAC778E,electric_bike,2020-09-17 15:09:00,2020-09-17 15:09:00,W Oakdale Ave & N Broadway,NaN,W Oakdale Ave & N Broadway,NaN,41.940000,-87.640000,41.940000,-87.640000,casual,0 days 00:00:00
3,57F6DC9A153DB98C,electric_bike,2020-09-17 18:10:00,2020-09-17 18:35:00,Ashland Ave & Belle Plaine Ave,246.0,Montrose Harbor,249.0,41.956060,-87.668916,41.963985,-87.638216,casual,0 days 00:25:00
4,B9C4712F78C1AE68,electric_bike,2020-09-17 15:16:00,2020-09-17 15:52:00,Fairbanks Ct & Grand Ave,24.0,Fairbanks Ct & Grand Ave,24.0,41.891860,-87.621008,41.891346,-87.620325,casual,0 days 00:36:00


In [29]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500684 entries, 0 to 532957
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype          
---  ------              --------------   -----          
 0   ride_id             500684 non-null  object         
 1   rideable_type       500684 non-null  object         
 2   started_at          500684 non-null  datetime64[ns] 
 3   ended_at            500684 non-null  datetime64[ns] 
 4   start_station_name  500684 non-null  object         
 5   start_station_id    500507 non-null  float64        
 6   end_station_name    500684 non-null  object         
 7   end_station_id      500546 non-null  float64        
 8   start_lat           500684 non-null  float64        
 9   start_lng           500684 non-null  float64        
 10  end_lat             500684 non-null  float64        
 11  end_lng             500684 non-null  float64        
 12  member_casual       500684 non-null  object         
 13  Duration           

In [30]:
df2[df2['Duration'] < pd.Timedelta('0 days')]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Duration


Split the started_at and ended_at into date and time to use it in visualiazations.

In [31]:
df2['start_date'] = df2['started_at'].dt.date
df2['start_time'] = df2['started_at'].dt.time

<ipython-input-31-62458e63cedb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['start_date'] = df2['started_at'].dt.date
<ipython-input-31-62458e63cedb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['start_time'] = df2['started_at'].dt.time


In [32]:
df2.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Duration,start_date,start_time
0,2B22BD5F95FB2629,electric_bike,2020-09-17 14:27:00,2020-09-17 14:44:00,Michigan Ave & Lake St,52.0,Green St & Randolph St,112.0,41.886692,-87.623561,41.883570,-87.648731,casual,0 days 00:17:00,2020-09-17,14:27:00
1,A7FB70B4AFC6CAF2,electric_bike,2020-09-17 15:07:00,2020-09-17 15:07:00,W Oakdale Ave & N Broadway,NaN,W Oakdale Ave & N Broadway,NaN,41.940000,-87.640000,41.940000,-87.640000,casual,0 days 00:00:00,2020-09-17,15:07:00
2,86057FA01BAC778E,electric_bike,2020-09-17 15:09:00,2020-09-17 15:09:00,W Oakdale Ave & N Broadway,NaN,W Oakdale Ave & N Broadway,NaN,41.940000,-87.640000,41.940000,-87.640000,casual,0 days 00:00:00,2020-09-17,15:09:00
3,57F6DC9A153DB98C,electric_bike,2020-09-17 18:10:00,2020-09-17 18:35:00,Ashland Ave & Belle Plaine Ave,246.0,Montrose Harbor,249.0,41.956060,-87.668916,41.963985,-87.638216,casual,0 days 00:25:00,2020-09-17,18:10:00
4,B9C4712F78C1AE68,electric_bike,2020-09-17 15:16:00,2020-09-17 15:52:00,Fairbanks Ct & Grand Ave,24.0,Fairbanks Ct & Grand Ave,24.0,41.891860,-87.621008,41.891346,-87.620325,casual,0 days 00:36:00,2020-09-17,15:16:00


In [33]:
df2['end_date'] = df2['ended_at'].dt.date
df2['end_time'] = df2['ended_at'].dt.time
df2.head(3)

<ipython-input-33-253860b68dcb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['end_date'] = df2['ended_at'].dt.date
<ipython-input-33-253860b68dcb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['end_time'] = df2['ended_at'].dt.time


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Duration,start_date,start_time,end_date,end_time
0,2B22BD5F95FB2629,electric_bike,2020-09-17 14:27:00,2020-09-17 14:44:00,Michigan Ave & Lake St,52.0,Green St & Randolph St,112.0,41.886692,-87.623561,41.88357,-87.648731,casual,0 days 00:17:00,2020-09-17,14:27:00,2020-09-17,14:44:00
1,A7FB70B4AFC6CAF2,electric_bike,2020-09-17 15:07:00,2020-09-17 15:07:00,W Oakdale Ave & N Broadway,NaN,W Oakdale Ave & N Broadway,NaN,41.940000,-87.640000,41.94000,-87.640000,casual,0 days 00:00:00,2020-09-17,15:07:00,2020-09-17,15:07:00
2,86057FA01BAC778E,electric_bike,2020-09-17 15:09:00,2020-09-17 15:09:00,W Oakdale Ave & N Broadway,NaN,W Oakdale Ave & N Broadway,NaN,41.940000,-87.640000,41.94000,-87.640000,casual,0 days 00:00:00,2020-09-17,15:09:00,2020-09-17,15:09:00


In [ ]:
df2.to_excel("data.xlsx")

In [43]:
import pandas as pd
import math

def haversine_distance(row):
    """
    Calculate the great-circle distance between two points
    on the Earth's surface using the Haversine formula.

    Parameters:
    row: DataFrame row containing start and end latitude and longitude.

    Returns:
    Distance between the two points in kilometers.
    """
    lat1, lon1 = row['start_lat'], row['start_lng']
    lat2, lon2 = row['end_lng'], row['end_lng']

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    radius_of_earth = 6371  # Earth's radius in kilometers
    distance = radius_of_earth * c

    return distance

# Apply the haversine_distance function to each row and create a new column 'ride_distance'
df2['ride_distance'] = df2.apply(haversine_distance, axis=1)




<ipython-input-43-ad7e1ac08ac2>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ride_distance'] = df2.apply(haversine_distance, axis=1)


In [44]:
df2.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Duration,start_date,start_time,end_date,end_time,ride_distance
0,2B22BD5F95FB2629,electric_bike,2020-09-17 14:27:00,2020-09-17 14:44:00,Michigan Ave & Lake St,52.0,Green St & Randolph St,112.0,41.886692,-87.623561,41.883570,-87.648731,casual,0 days 00:17:00,2020-09-17,14:27:00,2020-09-17,14:44:00,14403.681975
1,A7FB70B4AFC6CAF2,electric_bike,2020-09-17 15:07:00,2020-09-17 15:07:00,W Oakdale Ave & N Broadway,NaN,W Oakdale Ave & N Broadway,NaN,41.940000,-87.640000,41.940000,-87.640000,casual,0 days 00:00:00,2020-09-17,15:07:00,2020-09-17,15:07:00,14408.638595
2,86057FA01BAC778E,electric_bike,2020-09-17 15:09:00,2020-09-17 15:09:00,W Oakdale Ave & N Broadway,NaN,W Oakdale Ave & N Broadway,NaN,41.940000,-87.640000,41.940000,-87.640000,casual,0 days 00:00:00,2020-09-17,15:09:00,2020-09-17,15:09:00,14408.638595
3,57F6DC9A153DB98C,electric_bike,2020-09-17 18:10:00,2020-09-17 18:35:00,Ashland Ave & Belle Plaine Ave,246.0,Montrose Harbor,249.0,41.956060,-87.668916,41.963985,-87.638216,casual,0 days 00:25:00,2020-09-17,18:10:00,2020-09-17,18:35:00,14410.226161
4,B9C4712F78C1AE68,electric_bike,2020-09-17 15:16:00,2020-09-17 15:52:00,Fairbanks Ct & Grand Ave,24.0,Fairbanks Ct & Grand Ave,24.0,41.891860,-87.621008,41.891346,-87.620325,casual,0 days 00:36:00,2020-09-17,15:16:00,2020-09-17,15:52:00,14401.097836


In [47]:
df2['ride_distance'].unique()

array([14403.681975  , 14408.6385946 , 14410.22616094, ...,
       14385.59604057, 14413.87451072, 14403.97201924])

In [45]:
df2.to_excel("cs_with_distance.xlsx", sheet_name="Sheet1")